In [130]:
import string 
import numpy as np
import tensorflow as tf

In [131]:
with open ("dataset.txt", 'r') as f:
    data = f.read()
    f.close()
data = data.lower()
data = data.split(' ')

In [132]:
raw_paragraph = []
for text in data:
     raw_paragraph.append(''.join(x for x in text if x.isalpha()))
    
raw_paragraph[:10]

['after',
 'flying',
 'a',
 'long',
 'distance',
 'a',
 'thirsty',
 'crow',
 'was',
 'wandering']

In [133]:
unique_words = []
for word in raw_paragraph:
    if word not in unique_words:
        unique_words.append(word)
unique_words = np.array(unique_words)
unique_words

array(['after', 'flying', 'a', 'long', 'distance', 'thirsty', 'crow',
       'was', 'wandering', 'the', 'forest', 'in', 'search', 'of', 'water',
       'finally', 'he', 'saw', 'pot', 'halffilled', 'with', 'tried', 'to',
       'drink', 'from', 'it', 'but', 'his', 'beak', 'wasnt', 'enough',
       'reach', 'inside', 'then', 'pebbles', 'on', 'ground', 'and', 'one',
       'by', 'put', 'them', 'until', 'rose', 'brim', 'hastily', 'drank',
       'quenched', 'thirst'], dtype='<U10')

In [134]:
dict_size = len(unique_words)
paragraph_size = len(raw_paragraph)
encoding_mapping = {}

for word in raw_paragraph:
    encoding = np.zeros((dict_size))
    pos = np.where(unique_words == word)[0][0]
    encoding[pos] = 1
    encoding_mapping[word] = encoding

In [135]:
data = []
label =[]
window = [-2,-1,1,2]
for i in range(paragraph_size):
    working_word = raw_paragraph[i]
    for j in window:
        t = i + j
        if 0 <= t < paragraph_size:
            data.append(encoding_mapping[working_word])
            label.append(encoding_mapping[raw_paragraph[t]])
data = np.array(data)
label = np.array(label)

In [136]:
# np.save('data.npy', data)
# np.save('label.npy', label)

In [137]:
rng_state = np.random.get_state()
np.random.shuffle(data)
np.random.set_state(rng_state)
np.random.shuffle(label)

In [138]:
print(data.shape, label.shape)

(298, 49) (298, 49)


In [139]:
X_train = data[: int(0.9 * data.shape[0])]
y_train = label[: int(0.9 * label.shape[0])]

X_test = data[int(0.9 * data.shape[0]): ]
y_test = label[int(0.9 * label.shape[0]): ]

In [140]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(units = 150, activation = 'linear', input_shape = (49,), name='hidden'),
  tf.keras.layers.Dense(units = 49, activation = 'linear', name='output'),
])

In [141]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 150)               7500      
                                                                 
 output (Dense)              (None, 49)                7399      
                                                                 
Total params: 14,899
Trainable params: 14,899
Non-trainable params: 0
_________________________________________________________________


In [142]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [143]:
epochs = 60
# batch_size = 2

model.fit(data, label, epochs=epochs, validation_split=0.1, verbose = 0)

In [144]:
test = []

for word in raw_paragraph:
    test.append(encoding_mapping[word])
test = np.array(test)
predicted = model.predict(test)

out = "after"

for i, o in zip(test, predicted):
    # print(i, o)
    out += " " + unique_words[np.argmax(o)]
    # print(unique_words[np.argmax(i)], '->', unique_words[np.argmax(o)])
print(out)

3/3 [==============================] - 0s 2ms/step
after flying after distance a a distance a the thirsty the in search the of in he water water he distance with a pot he water drink the from it from from beak wasnt long a wasnt the water in he then water he he the pebbles in the ground by one by water in in the in with the in he the the in the in the he crow then it from ground it beak quenched
